### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Login: fabin99
Password: ········
Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
20,38890279,10528,79085,51531,0,0,973,2,2022-02-28 22:00,2022-02-28 22:00,0,2,2730,4,1
21,38890318,90579,79085,51530,0,0,973,3,2022-03-01 19:00,2022-03-01 19:00,1,0,1539,2,1
22,38889190,105120,79085,3,0,0,973,4,2022-03-02 18:27,2022-03-02 18:29,5,1,414,5,1
23,38890203,121094,79085,51530,0,0,973,5,2022-03-03 22:00,2022-03-03 22:00,6,0,3921,5,1
24,38890912,121339,79085,51531,0,0,973,6,2022-03-04 22:00,2022-03-04 22:00,2,0,1536,5,1
25,38893440,11244,79085,51531,0,0,974,0,2022-03-06 00:00,2022-03-06 00:00,0,1,1895,1,1
26,38705572,79085,6335,2074,14,55,974,1,2022-03-06 21:58,2022-03-06 21:59,1,0,3100,5,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

122

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,10528,44113,29444,0,3,1,0,0,4-4-2,5,5,5,38890279
1,79085,25866,40535,4,9,3,0,0,5-2-3e,7,7,7,38890279
0,90579,26987,38205,1,5,1,0,0,d.po 2,9,9,9,38890318
1,79085,42969,31751,4,4,1,1,0,5-2-3f,7,7,7,38890318
0,105120,22368,43820,0,11,1,0,0,Hedef,8,7,5,38889190
1,79085,47663,26211,0,2,7,2,0,5-2-3e,7,5,7,38889190
0,121094,21838,39613,1,17,3,2,0,4132,11,10,8,38890203
1,79085,48343,30568,1,3,5,2,1,5-2-3f,7,6,7,38890203
0,121339,39333,31380,2,6,3,2,0,Nada normal,6,8,7,38890912
1,79085,30867,38820,1,12,2,0,0,3-4-3a,8,7,6,38890912


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

244

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 2074

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,79085,14,40,13,1,0,79,0,40579079013008,2074
1,6335,14,37,12,1,1,63,4,37559063012007,2074
2,10467,14,28,9,1,4,56,20,28536056009005,2074
3,121325,14,25,8,1,5,39,29,25510039008004,2074
4,89684,14,18,6,0,8,39,39,18500039006006,2074
5,121670,14,12,4,0,10,10,60,12450010004003,2074
6,78774,14,3,1,0,13,4,63,3441004001001,2074
7,48360,14,3,1,0,13,8,83,3425008001002,2074


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

1

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()